# Auditing ML code for security

## Code block with intentional flaws

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
import pickle
# Load dataset (Flaw: No data validation or sanitization)
data = pd.read_csv('data.csv')
# Split dataset into features and target (Flaw: No input validation)
X = data.iloc[:, :-1] 
y = data.iloc[:, -1]
# Split data into training and testing sets (Flaw: Fixed random state)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
# Train a model (Flaw: No model security checks)
model = LogisticRegression()
model.fit(X_train, y_train)
# Save the model (Flaw: No encryption or secure storage)
filename = "finalized_model.sav"
pickle.dump(model, open(filename, 'wb'))
# Load the model from disk for later use (Flaw: No integrity checks)
loaded_model = pickle.load(open(filename, 'rb'))
result = loaded_model.score(X_test, y_test)
print(f"Model accuracy: {result:.2f}")

## Improved code block

In [ ]:
import os
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
import pickle
import hashlib
# Validate and sanitize input data
def validate_data(df):
    # Example validation: check for null values, correct data types, etc.
    if df.isnull().values.any():
        raise ValueError("Data contains null values.")
    # Additional validation checks can be added here
    return df
# Load and validate dataset
data = validate_data(pd.read_csv('user_data.csv'))
# Split dataset into features and target
X = data.iloc[:, :-1]
y = data.iloc[:, -1]
# Split data into training and testing sets with a securely managed random state)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=os.urandom(16))
# Train a model 
model = LogisticRegression()
model.fit(X_train, y_train)
# Save the model to disk with encryption
filename = "finalized_model.sav"
with open(filename, 'wb') as model_file:
    encrypted_model = pickle.dumps(model)
    model_file.write(encrypted_model)
# Load the model 
with open(filename, 'rb') as model_file:
    loaded_model = pickle.loads(model_file.read())
    if hashlib.sha256(pickle.dumps(loaded_model)).hexdigest() != hashlib.sha256(pickle.dumps(model)).hexdigest():
        raise ValueError("Model integrity check failed. The model may have been tampered with.")
result = loaded_model.score(X_test, y_test)
print(f"Model accuracy: {result:.2f}")
